In [3]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=6984c4302c13c8f6158973b7e4f9184890f4c939a73e34b2c9ef054f59086546
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [4]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=TotjYe4BYcMGvQP4QIVVq3g0PpvDji&prompt=consent&token_usage=remote&access_type=offline&code_challenge=kkXy8JbtGO-8awnNbPv-LSS6VpHWsEZxcratKpF11ho&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrBBN32CeZPoOtyEFD69E7iejDS09u97DhaK5R9WLSeokyrnCHRTv_pfI5u-xvBzJw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [5]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [6]:
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `repositoriodedadosgpsp.Datalake.CNES_vinculo_saude_tipo_ocupacao_v2` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"
df = client.query(query).to_dataframe()
df

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos
0,2008,Profissionais da fisioterapia e afins,AP,1600600,2
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400159,46
2,2008,Farmacêuticos,RR,1400472,2
3,2008,Técnicos de odontologia,DF,5300108,11
4,2008,Técnicos e auxiliares de enfermagem,DF,5300108,3238
...,...,...,...,...,...
1836885,2024,Técnicos e auxiliares de enfermagem,TO,1712009,15
1836886,2024,Engenheiros civis e afins,TO,1721000,2
1836887,2024,Biomédicos,TO,1712157,1
1836888,2024,Gestores e especialistas de operações em empre...,TO,1700301,2


In [7]:
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.startswith('Médicos'),'Médicos',df['tipo_ocupacao'])
## Aqui eu unifico os três tipos de Médicos (clínica, cirurgia e terapeutico)
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
0,2008,Profissionais da fisioterapia e afins,AP,1600600,2,Profissionais da fisioterapia e afins
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400159,46,Trabalhadores em serviços de promoção e apoio ...
2,2008,Farmacêuticos,RR,1400472,2,Farmacêuticos
3,2008,Técnicos de odontologia,DF,5300108,11,Técnicos de odontologia
4,2008,Técnicos e auxiliares de enfermagem,DF,5300108,3238,Técnicos e auxiliares de enfermagem
...,...,...,...,...,...,...
1836885,2024,Técnicos e auxiliares de enfermagem,TO,1712009,15,Técnicos e auxiliares de enfermagem
1836886,2024,Engenheiros civis e afins,TO,1721000,2,Engenheiros civis e afins
1836887,2024,Biomédicos,TO,1712157,1,Biomédicos
1836888,2024,Gestores e especialistas de operações em empre...,TO,1700301,2,Gestores e especialistas de operações em empre...


In [8]:
#Alternando range
df = df[(df['tipo_2']=='Médicos') & (df['ano']>2017) & (df['ano']<=2023) ]
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
731636,2018,Médicos clínicos,RR,1400704,11,Médicos
731638,2018,Médicos clínicos,RR,1400233,7,Médicos
731649,2018,Médicos clínicos,AP,1600105,5,Médicos
731651,2018,Médicos em especialidades cirúrgicas,RR,1400456,1,Médicos
731672,2018,Médicos clínicos,RR,1400027,7,Médicos
...,...,...,...,...,...,...
1786994,2023,Médicos em medicina diagnóstica e terapêutica,TO,1716505,5,Médicos
1787022,2023,Médicos clínicos,TO,1707108,13,Médicos
1787044,2023,Médicos em especialidades cirúrgicas,TO,1701903,4,Médicos
1787055,2023,Médicos clínicos,TO,1712504,5,Médicos


In [10]:
## Chamando a quantidade de população.

pop = pd.read_csv("POP_CD2022_ATT.csv");
pop

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POP. COLETADA,POP. IMPUTADA,POPULAÇÃO,id_municipio
0,RO,11,15,Alta Floresta D'Oeste,20865,629,21494,1100015
1,RO,11,23,Ariquemes,93006,3827,96833,1100023
2,RO,11,31,Cabixi,5100,251,5351,1100031
3,RO,11,49,Cacoal,84923,1964,86887,1100049
4,RO,11,56,Cerejeiras,15297,593,15890,1100056
...,...,...,...,...,...,...,...,...
5565,GO,52,22005,Vianópolis,14345,611,14956,5222005
5566,GO,52,22054,Vicentinópolis,8592,176,8768,5222054
5567,GO,52,22203,Vila Boa,4168,47,4215,5222203
5568,GO,52,22302,Vila Propício,5594,221,5815,5222302


In [11]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
731636,2018,Médicos clínicos,RR,1400704,11,Médicos
731638,2018,Médicos clínicos,RR,1400233,7,Médicos
731649,2018,Médicos clínicos,AP,1600105,5,Médicos
731651,2018,Médicos em especialidades cirúrgicas,RR,1400456,1,Médicos
731672,2018,Médicos clínicos,RR,1400027,7,Médicos
731683,2018,Médicos clínicos,AP,1600402,12,Médicos
731690,2018,Médicos em especialidades cirúrgicas,RR,1400472,18,Médicos
731717,2018,Médicos clínicos,AC,1200393,7,Médicos
731733,2018,Médicos clínicos,AC,1200609,22,Médicos
731771,2018,Médicos em especialidades cirúrgicas,AL,2702801,1,Médicos


In [12]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

<ipython-input-12-a8331f5d24a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_municipio'] = pd.to_numeric(df['id_municipio'])


In [13]:
x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

<ipython-input-13-26384f7e5ec1>:1: FutureWarning: The provided callable <function sum at 0x79fc92d02dd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)


In [14]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [15]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [16]:
sum(df1[df1['ano']==2023]['POPULAÇÃO'])

203080756

In [17]:
sum(df1[df1['ano']==2023]['quantidade_vinculos'])/sum(df1[df1['ano']==2023]['POPULAÇÃO'])*1000

2.752963948981951

In [18]:
## validando alguns dados
df1[(df1['ano']==2023) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
21950,2023,RJ,3304508,Médicos,84,RJ,Rio das Flores,8954,9.381282
21908,2023,RJ,3304110,Médicos,151,RJ,Porto Real,20373,7.41177
21944,2023,RJ,3304409,Médicos,128,RJ,Rio Claro,17401,7.355899
21890,2023,RJ,3303955,Médicos,163,RJ,Pinheiral,24298,6.708371
21998,2023,RJ,3305000,Médicos,243,RJ,São João da Barra,36573,6.644246
21794,2023,RJ,3302601,Médicos,247,RJ,Mangaratiba,41220,5.992237
21584,2023,RJ,3300233,Médicos,226,RJ,Armação dos Búzios,40006,5.649153
22106,2023,RJ,3306305,Médicos,1463,RJ,Volta Redonda,261563,5.593299
22040,2023,RJ,3305505,Médicos,479,RJ,Saquarema,89559,5.34843
21590,2023,RJ,3300258,Médicos,159,RJ,Arraial do Cabo,30986,5.13135


In [19]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [20]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [21]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_ocupacao',
       'quantidade_vinculos', 'nome_municipio', 'populacao_domiciliada',
       'taxa'],
      dtype='object')

In [22]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',
       'taxa']]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33411 entries, 32079 to 16705
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    33411 non-null  Int64  
 1   sigla_uf               33411 non-null  object 
 2   nome_municipio         33411 non-null  object 
 3   id_municipio           33411 non-null  int64  
 4   populacao_domiciliada  33411 non-null  int64  
 5   tipo_ocupacao          33411 non-null  object 
 6   quantidade_vinculos    33411 non-null  Int64  
 7   taxa                   33411 non-null  Float64
dtypes: Float64(1), Int64(2), int64(2), object(3)
memory usage: 2.4+ MB


In [24]:

schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','Float',description='Taxa de médicos por 1000 habitantes')
 ]


In [25]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [26]:
table_ref = dataset_ref.table('CNES_medicos_mil_habitantes_v3') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=b9de6ff9-dbfb-43fe-a55a-389053567305>